In [1]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# !pip install -U PyDrive

In [3]:
#Import authentication libraries

# from google.colab import auth
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from oauth2client.client import GoogleCredentials

In [4]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [5]:
# folder_id='1uXkcRXuQ5fFG0S8mLW67ewj_jsqAb_Ff'

In [6]:
# # Auto-iterate through all files in the midi folder.
# file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(folder_id)}).GetList()
# for file1 in file_list:
#   print('title: {0}, id: {1}'.format(file1['title'], file1['id']))

In [7]:
# # Download all midi files
# for file1 in file_list:
#   file_id=file1['id']
#   file_title = file1['title']
#   drive.CreateFile({'id': file_id}).GetContentFile(file_title)
  

In [8]:
# from os import listdir
# from os.path import isfile, join

In [9]:
# echo %cd%

In [10]:
# folder ='C:/Users/Gustavo/Documents/GitHub/MidiNet_test/cond2d'
# 

In [11]:
# songs=[f for f in listdir(folder)]
# print(songs)
# songs=songs[-10:-2]

In [12]:
# songs

In [13]:
# data_songs=np.empty((0,16,128), int)
# for song in songs:
#   song_opened = open(song,"rb")
#   song_loaded= pickle.load(song_opened)
#   #for track 1
#   for i in range(len(song_loaded[0])):
#     bar=np.empty((0,128), int)[None]
#     for j in song_loaded[0][i]:
#       note=np.zeros(128)[None][None]
#       if type(j) == type(1.5):
#         note[0,0,int(j)] = 1
#       bar=np.append(bar,note,axis=1)
#     data_songs =np.append(data_songs,bar, axis=0)
#   print(np.array(data_songs).shape)
# data_songs_prev =np.roll(data_songs,1,axis=0)
# data_songs=data_songs[None]
# data_songs_prev=data_songs_prev[None]

In [14]:
# print(data_songs.shape)
# print(data_songs_prev.shape)
# data_songs=np.transpose(data_songs, (1,0,2,3))#Real data, input for model
# data_songs_prev=np.transpose(data_songs_prev, (1,0,2,3))#Real data, 1 previous bar, input for model
# print(data_songs.shape)
# print(data_songs_prev.shape)

In [15]:
# np.save('data_songs.npy', data_songs, allow_pickle=True, fix_imports=True)

In [16]:
# np.save('data_songs_prev.npy', data_songs_prev, allow_pickle=True, fix_imports=True)

In [17]:
import os
import scipy.misc
import numpy as np
from model import MidiNet #Llamando el modelo
from utils import pp, to_json, generation_test

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 20, "Epoch to train [20]")
flags.DEFINE_float("learning_rate", 0.00005, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("batch_size", 12, "The size of batch [12]")
flags.DEFINE_integer("output_w", 16, "The size of the output segs to produce [16]")
flags.DEFINE_integer("output_h", 128, "The size of the output note to produce [128]")
flags.DEFINE_integer("c_dim", 1, "Number of Midi track. [1]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Dircacectory name to save the image samples [samples]")
flags.DEFINE_string("dataset", "MidiNet_vg", "The name of dataset ")
flags.DEFINE_boolean("is_train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("is_crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("generation_test", False, "True for generation_test, False for nothing [False]")
flags.DEFINE_string("gen_dir", "gen", "Directory name to save the generate samples [samples]")
FLAGS = flags.FLAGS

def main(_):
    pp.pprint(flags.FLAGS.__flags)

    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)
    if not os.path.exists(FLAGS.gen_dir):
        os.makedirs(FLAGS.gen_dir)

    with tf.Session() as sess:
        if FLAGS.dataset == 'MidiNet_vg':
            model = MidiNet(sess, batch_size=FLAGS.batch_size, output_w=FLAGS.output_w, output_h=FLAGS.output_h, c_dim=FLAGS.c_dim,
                    dataset_name=FLAGS.dataset, is_crop=FLAGS.is_crop, checkpoint_dir=FLAGS.checkpoint_dir, sample_dir=FLAGS.sample_dir, 
                            gen_dir=FLAGS.gen_dir)
        
        if FLAGS.is_train:
            model.train(FLAGS)
        else:
            model.load(FLAGS.checkpoint_dir)

        

if __name__ == '__main__':
    tf.app.run()

{'batch_size': <absl.flags._flag.Flag object at 0x00000275C9AC1CC0>,
 'beta1': <absl.flags._flag.Flag object at 0x00000275C9A32400>,
 'c_dim': <absl.flags._flag.Flag object at 0x00000275C9AC1DD8>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x00000275C9AC1E80>,
 'dataset': <absl.flags._flag.Flag object at 0x00000275C9ACF0B8>,
 'epoch': <absl.flags._flag.Flag object at 0x00000275C9A32278>,
 'gen_dir': <absl.flags._flag.Flag object at 0x00000275C9ACF208>,
 'generation_test': <absl.flags._flag.BooleanFlag object at 0x00000275C9ACF240>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x00000275C9ACF320>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x00000275C9ACF320>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x00000275C9ACF3C8>,
 'helpshort': <tensorflow.python.platform.app._HelpshortFlag object at 0x00000275C9ACF438>,
 'is_crop': <absl.flags._flag.BooleanFlag object at 0x00000275C9ACF198>,
 'is_train': <absl.flags._flag.Boolea

Epoch: [ 1] [  23/  65] time: 13.4130, d_loss: 0.96400893, g_loss: 32.56191635
Epoch: [ 1] [  24/  65] time: 13.5367, d_loss: 0.92094451, g_loss: 26.63718987
Epoch: [ 1] [  25/  65] time: 13.6598, d_loss: 0.90921724, g_loss: 28.84046936
Epoch: [ 1] [  26/  65] time: 13.7800, d_loss: 0.99273276, g_loss: 23.34719467
Epoch: [ 1] [  27/  65] time: 13.9032, d_loss: 0.87076306, g_loss: 19.09993362
Epoch: [ 1] [  28/  65] time: 14.0221, d_loss: 0.92167342, g_loss: 16.76830673
Epoch: [ 1] [  29/  65] time: 14.1437, d_loss: 0.95569199, g_loss: 28.61331940
Epoch: [ 1] [  30/  65] time: 14.2634, d_loss: 0.91935182, g_loss: 24.42091179
Epoch: [ 1] [  31/  65] time: 14.3881, d_loss: 0.86680686, g_loss: 20.63483620
Epoch: [ 1] [  32/  65] time: 14.5277, d_loss: 0.85680854, g_loss: 16.90356827
Epoch: [ 1] [  33/  65] time: 14.6653, d_loss: 0.91820931, g_loss: 25.11002731
Epoch: [ 1] [  34/  65] time: 14.7980, d_loss: 0.82327724, g_loss: 20.36264801
Epoch: [ 1] [  35/  65] time: 14.9197, d_loss: 0.872

Epoch: [ 2] [  61/  65] time: 27.3964, d_loss: 0.59995711, g_loss: 26.73293114
Epoch: [ 2] [  62/  65] time: 27.5400, d_loss: 0.62251854, g_loss: 20.45234108
Epoch: [ 2] [  63/  65] time: 27.6747, d_loss: 0.63658196, g_loss: 17.17839241
Epoch: [ 2] [  64/  65] time: 27.8143, d_loss: 0.56991869, g_loss: 22.76677513
Epoch: [ 2] time: 28.1002, d_loss: 0.00876798
Epoch: [ 3] [   0/  65] time: 28.2359, d_loss: 0.62019920, g_loss: 18.19151497
Epoch: [ 3] [   1/  65] time: 28.3665, d_loss: 0.66533256, g_loss: 26.88709641
Epoch: [ 3] [   2/  65] time: 28.5057, d_loss: 0.63230550, g_loss: 19.31625366
Epoch: [ 3] [   3/  65] time: 28.6473, d_loss: 0.59579706, g_loss: 21.61120796
Epoch: [ 3] [   4/  65] time: 28.7730, d_loss: 0.59552032, g_loss: 23.69110489
Epoch: [ 3] [   5/  65] time: 28.8996, d_loss: 0.65364230, g_loss: 24.55069351
[Sample] d_loss: 0.64108181, g_loss: 22.33383369
Epoch: [ 3] [   6/  65] time: 29.0495, d_loss: 0.58869106, g_loss: 30.09072685
Epoch: [ 3] [   7/  65] time: 29.178

Epoch: [ 4] [  33/  65] time: 41.5293, d_loss: 0.49347076, g_loss: 23.94614029
Epoch: [ 4] [  34/  65] time: 41.6749, d_loss: 0.49308440, g_loss: 18.99197388
Epoch: [ 4] [  35/  65] time: 41.8115, d_loss: 0.49021128, g_loss: 18.11114693
Epoch: [ 4] [  36/  65] time: 41.9302, d_loss: 0.49128830, g_loss: 18.19693565
Epoch: [ 4] [  37/  65] time: 42.0529, d_loss: 0.49227834, g_loss: 20.51382065
Epoch: [ 4] [  38/  65] time: 42.1745, d_loss: 0.48553360, g_loss: 19.19707108
Epoch: [ 4] [  39/  65] time: 42.3101, d_loss: 0.49086350, g_loss: 20.30519485
Epoch: [ 4] [  40/  65] time: 42.4547, d_loss: 0.53789353, g_loss: 22.48392487
[Sample] d_loss: 0.48982495, g_loss: 24.16980743
Epoch: [ 4] [  41/  65] time: 42.6107, d_loss: 0.51332182, g_loss: 18.71479416
Epoch: [ 4] [  42/  65] time: 42.7284, d_loss: 0.50617313, g_loss: 25.77227020
Epoch: [ 4] [  43/  65] time: 42.8610, d_loss: 0.51156127, g_loss: 21.00856972
Epoch: [ 4] [  44/  65] time: 42.9797, d_loss: 0.47064543, g_loss: 17.61744690
Epo

Epoch: [ 6] [   5/  65] time: 55.5131, d_loss: 0.47995996, g_loss: 22.43492126
Epoch: [ 6] [   6/  65] time: 55.6337, d_loss: 0.40741536, g_loss: 27.71228027
Epoch: [ 6] [   7/  65] time: 55.7574, d_loss: 0.45034727, g_loss: 21.48563957
Epoch: [ 6] [   8/  65] time: 55.8761, d_loss: 0.43678150, g_loss: 22.81229019
Epoch: [ 6] [   9/  65] time: 55.9958, d_loss: 0.41840020, g_loss: 27.35312462
Epoch: [ 6] [  10/  65] time: 56.1217, d_loss: 0.43470460, g_loss: 17.84700012
[Sample] d_loss: 0.43414989, g_loss: 22.82363701
Epoch: [ 6] [  11/  65] time: 56.2942, d_loss: 0.42126262, g_loss: 21.30150795
Epoch: [ 6] [  12/  65] time: 56.4259, d_loss: 0.42287529, g_loss: 22.01254463
Epoch: [ 6] [  13/  65] time: 56.5546, d_loss: 0.43408462, g_loss: 19.61080742
Epoch: [ 6] [  14/  65] time: 56.6802, d_loss: 0.39781374, g_loss: 18.18718910
Epoch: [ 6] [  15/  65] time: 56.8089, d_loss: 0.39857793, g_loss: 21.92408371
Epoch: [ 6] [  16/  65] time: 56.9305, d_loss: 0.42659521, g_loss: 16.37944603
Epo

...

Epoch: [15] [  35/  65] time: 137.1957, d_loss: 0.38658324, g_loss: 15.72499466
Epoch: [15] [  36/  65] time: 137.3182, d_loss: 0.39977434, g_loss: 15.80173111
Epoch: [15] [  37/  65] time: 137.4428, d_loss: 0.42892009, g_loss: 17.48273277
Epoch: [15] [  38/  65] time: 137.5695, d_loss: 0.44954562, g_loss: 17.39378357
Epoch: [15] [  39/  65] time: 137.7071, d_loss: 0.40079680, g_loss: 18.59896088
Epoch: [15] [  40/  65] time: 137.8298, d_loss: 0.42625374, g_loss: 20.11681938
Epoch: [15] [  41/  65] time: 137.9555, d_loss: 0.41635790, g_loss: 15.84266949
Epoch: [15] [  42/  65] time: 138.0761, d_loss: 0.43452001, g_loss: 20.69645309
Epoch: [15] [  43/  65] time: 138.1998, d_loss: 0.42817387, g_loss: 18.76858330
Epoch: [15] [  44/  65] time: 138.3195, d_loss: 0.42786005, g_loss: 14.27725601
Epoch: [15] [  45/  65] time: 138.4422, d_loss: 0.40766734, g_loss: 19.37656593
Epoch: [15] [  46/  65] time: 138.5638, d_loss: 0.38166815, g_loss: 21.28320312
Epoch: [15] [  47/  65] time: 138.6845, 

Epoch: [17] [   7/  65] time: 150.9830, d_loss: 0.43474302, g_loss: 18.35487938
Epoch: [17] [   8/  65] time: 151.1287, d_loss: 0.50990564, g_loss: 15.66164303
Epoch: [17] [   9/  65] time: 151.2743, d_loss: 0.47873107, g_loss: 21.90508842
Epoch: [17] [  10/  65] time: 151.4109, d_loss: 0.43874684, g_loss: 15.52432823
Epoch: [17] [  11/  65] time: 151.5461, d_loss: 0.48025358, g_loss: 15.73689270
Epoch: [17] [  12/  65] time: 151.6747, d_loss: 0.44921431, g_loss: 19.22899628
Epoch: [17] [  13/  65] time: 151.7994, d_loss: 0.47614384, g_loss: 17.28978348
Epoch: [17] [  14/  65] time: 151.9201, d_loss: 0.43017268, g_loss: 15.03283119
Epoch: [17] [  15/  65] time: 152.0428, d_loss: 0.48145664, g_loss: 14.85789204
Epoch: [17] [  16/  65] time: 152.1664, d_loss: 0.55170661, g_loss: 12.45056534
Epoch: [17] [  17/  65] time: 152.2951, d_loss: 0.63793147, g_loss: 15.14338398
Epoch: [17] [  18/  65] time: 152.4327, d_loss: 0.44940510, g_loss: 14.83452034
Epoch: [17] [  19/  65] time: 152.5634, 

Epoch: [18] [  45/  65] time: 164.8018, d_loss: 0.40908766, g_loss: 19.16293335
Epoch: [18] [  46/  65] time: 164.9284, d_loss: 0.39252847, g_loss: 20.15127182
Epoch: [18] [  47/  65] time: 165.0591, d_loss: 0.46905577, g_loss: 15.06526852
Epoch: [18] [  48/  65] time: 165.1827, d_loss: 0.43152630, g_loss: 19.32790375
Epoch: [18] [  49/  65] time: 165.3085, d_loss: 0.44921356, g_loss: 16.36771393
Epoch: [18] [  50/  65] time: 165.4351, d_loss: 0.49151897, g_loss: 15.52157211
Epoch: [18] [  51/  65] time: 165.5670, d_loss: 0.46366256, g_loss: 14.28037262
Epoch: [18] [  52/  65] time: 165.7002, d_loss: 0.46578825, g_loss: 18.34625626
Epoch: [18] [  53/  65] time: 165.8308, d_loss: 0.43677682, g_loss: 15.76494026
Epoch: [18] [  54/  65] time: 165.9555, d_loss: 0.42315647, g_loss: 16.07725525
Epoch: [18] [  55/  65] time: 166.0832, d_loss: 0.44333407, g_loss: 21.64037704
Epoch: [18] [  56/  65] time: 166.2128, d_loss: 0.53792673, g_loss: 20.10091209
Epoch: [18] [  57/  65] time: 166.3405, 

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
